In [ ]:
import pyspark
import os
import socket
import random

In [ ]:
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3.6'

In [ ]:
conf = pyspark.conf.SparkConf()

In [ ]:
k8sMaster = "https://10.0.0.1:443"

In [1]:
# Azure storage access info
blob_account_name = "azureopendatastorage"
blob_container_name = "censusdatacontainer"
blob_relative_path = "release/us_population_zip/"
blob_sas_token = r""

In [ ]:
# Allow SPARK to read from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
print('Remote blob path: ' + wasbs_path)

In [ ]:
conf = conf.setMaster("k8s://" + k8sMaster) \
    .setAppName("pyspark-notebook") \
    .set("spark.kubernetes.executor.container.image", "docker.io/guislander/spark-py:latest")  \
    .set("spark.kubernetes.namespace", "jhub") \
    .set("spark.kubernetes.authenticate.driver.serviceAccountName","spark")  \
    .set("spark.executor.instances", "2") \
    .set("spark.kubernetes.executor.request.cores","100m") \
    .set("spark.kubernetes.executor.limit.cores","1") \
    .set("spark.kubernetes.pyspark.pythonVersion","3") \
    .set("fs.azure.sas.%s.%s.blob.core.windows.net" % (blob_container_name, blob_account_name),
  blob_sas_token) \
    .set("spark.driver.host", socket.gethostbyaddr(socket.gethostname())[2][0]) \
    .set("spark.driver.port", "29413") \
    .set("spark.driver.pod.name", socket.gethostname())

In [3]:
# SPARK read parquet, note that it won't load any data yet by now
df = spark.read.parquet(wasbs_path)
print('Register the DataFrame as a SQL temporary view: source')
df.createOrReplaceTempView('source')

In [4]:
# Display top 10 rows
print('Displaying top 10 rows: ')
display(spark.sql('SELECT * FROM source LIMIT 10'))

In [ ]:
sc.stop()